In [1]:
import pandas as pd
import numpy as np

import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import LinearSVC

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve
import matplotlib.pyplot as plt

In [2]:
all_data = pd.read_csv("../output_files/precalculus_pairs_data.csv")
pred_tfidf_val = pd.read_csv("data/predicted_prereq.csv")
rank_val = pd.read_csv("data/concept_rank.csv")

In [3]:
all_data

,Unnamed: 0,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f31,f32,f33,f34,f35,f36,f37,relation,topic_a,topic_b
0,0,257.0,349.0,797.0,583.0,1.0,3.0,11.0,0.0,0.0,...,0.000000,352.0,831.0,50.0,-0.000040,0.000014,-7.542200e-07,1.0,Differential equation,Number
1,1,55.0,197.0,149.0,286.0,2.0,3.0,10.0,1.0,1.0,...,0.000000,289.0,374.0,61.0,0.000009,-0.000007,-1.910268e-06,1.0,Asymptote,Line (geometry)
2,2,141.0,197.0,621.0,286.0,5.0,3.0,18.0,1.0,1.0,...,0.000000,555.0,374.0,79.0,0.000018,0.000021,6.730535e-06,1.0,Cartesian coordinate system,Line (geometry)
3,3,57.0,198.0,28.0,660.0,3.0,2.0,6.0,0.0,0.0,...,0.000000,278.0,524.0,36.0,-0.000045,-0.000055,-6.749430e-06,1.0,Partial fraction decomposition,Arithmetic
4,4,370.0,102.0,614.0,295.0,3.0,3.0,44.0,2.0,14.0,...,0.023195,918.0,269.0,119.0,0.000051,0.000017,2.138242e-06,0.0,Logarithm,Natural logarithm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,2055,202.0,185.0,838.0,1088.0,2.0,5.0,17.0,0.0,2.0,...,0.000000,580.0,319.0,55.0,-0.000038,-0.000011,-2.839315e-07,0.0,Polynomial,Natural number
2056,2056,442.0,20.0,15414.0,55.0,4.0,3.0,4.0,0.0,0.0,...,0.000000,862.0,97.0,7.0,0.000066,0.001274,1.016967e-04,0.0,Mathematics,Polynomial long division
2057,2057,146.0,64.0,399.0,86.0,4.0,2.0,8.0,0.0,0.0,...,0.000853,326.0,141.0,23.0,0.000270,0.000015,3.585556e-06,0.0,Multiplication,Minor (linear algebra)
2058,2058,101.0,363.0,145.0,3440.0,2.0,1.0,4.0,0.0,0.0,...,0.000000,45.0,665.0,3.0,-0.000051,-0.000227,-2.401727e-05,1.0,Trigonometric substitution,Geometry


In [4]:
all_data = pd.merge(all_data, pred_tfidf_val, how = 'inner', on = ['topic_a', 'topic_b', 'relation'])
all_data

,Unnamed: 0_x,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f34,f35,f36,f37,relation,topic_a,topic_b,Unnamed: 0_y,tfidf_score,wiki_tfidf_score
0,0,257.0,349.0,797.0,583.0,1.0,3.0,11.0,0.0,0.0,...,50.0,-0.000040,0.000014,-7.542200e-07,1.0,Differential equation,Number,0,0.006426,0.006426
1,3,57.0,198.0,28.0,660.0,3.0,2.0,6.0,0.0,0.0,...,36.0,-0.000045,-0.000055,-6.749430e-06,1.0,Partial fraction decomposition,Arithmetic,1,0.047547,0.000000
2,8,193.0,157.0,576.0,182.0,2.0,3.0,24.0,2.0,2.0,...,62.0,-0.000040,0.000003,2.435413e-06,0.0,Equation,Quadratic equation,2,0.186873,0.186873
3,9,442.0,20.0,15414.0,14.0,4.0,3.0,3.0,0.0,0.0,...,19.0,0.000066,0.001275,1.021490e-04,0.0,Mathematics,Linear inequality,3,0.000000,0.000000
4,10,108.0,255.0,131.0,293.0,2.0,4.0,14.0,0.0,0.0,...,56.0,0.000016,-0.000005,-2.538441e-06,1.0,Binomial theorem,Addition,4,0.035118,0.035118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,2048,51.0,442.0,234.0,15414.0,6.0,4.0,10.0,0.0,1.0,...,23.0,-0.000061,-0.001267,-9.993580e-05,1.0,Limit (mathematics),Mathematics,913,1.000000,1.000000
914,2050,151.0,79.0,267.0,114.0,1.0,6.0,37.0,0.0,1.0,...,52.0,-0.000052,0.000011,1.894251e-06,0.0,Sine,Inverse trigonometric functions,914,0.000000,0.000000
915,2051,157.0,144.0,182.0,690.0,3.0,4.0,12.0,0.0,1.0,...,31.0,0.000016,-0.000017,-5.094307e-06,0.0,Quadratic equation,Rational number,915,0.022569,0.022569
916,2053,82.0,198.0,207.0,660.0,3.0,2.0,14.0,1.0,0.0,...,27.0,-0.000054,-0.000047,-5.103651e-06,1.0,Summation,Arithmetic,916,0.047547,0.047547


In [5]:
concept_rank = {}

for i in range(rank_val.shape[0]):
    concept = rank_val[["concept"]].iloc[i].values[0]
    rank = rank_val[["rank"]].iloc[i].values[0]
    concept_rank[concept] = rank

In [6]:
def rank_score(c1, c2):
    r1 = int(concept_rank[c1])
    r2 = int(concept_rank[c2])
    if r1 <= r2:
        return (r2-r1)
    else:
        return (r2-r1)

In [7]:
pred_rank = []
for i in range(all_data.shape[0]):
    topic_a = all_data[["topic_a"]].iloc[i].values[0]
    topic_b = all_data[["topic_b"]].iloc[i].values[0]
    tfidf_score = all_data[["tfidf_score"]].iloc[i].values[0]
    rank = rank_score(topic_a, topic_b)
    pred_rank.append(rank)

In [8]:
all_data["rank"] = pred_rank

In [9]:
all_data

,Unnamed: 0_x,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f35,f36,f37,relation,topic_a,topic_b,Unnamed: 0_y,tfidf_score,wiki_tfidf_score,rank
0,0,257.0,349.0,797.0,583.0,1.0,3.0,11.0,0.0,0.0,...,-0.000040,0.000014,-7.542200e-07,1.0,Differential equation,Number,0,0.006426,0.006426,-4
1,3,57.0,198.0,28.0,660.0,3.0,2.0,6.0,0.0,0.0,...,-0.000045,-0.000055,-6.749430e-06,1.0,Partial fraction decomposition,Arithmetic,1,0.047547,0.000000,-29
2,8,193.0,157.0,576.0,182.0,2.0,3.0,24.0,2.0,2.0,...,-0.000040,0.000003,2.435413e-06,0.0,Equation,Quadratic equation,2,0.186873,0.186873,0
3,9,442.0,20.0,15414.0,14.0,4.0,3.0,3.0,0.0,0.0,...,0.000066,0.001275,1.021490e-04,0.0,Mathematics,Linear inequality,3,0.000000,0.000000,-3
4,10,108.0,255.0,131.0,293.0,2.0,4.0,14.0,0.0,0.0,...,0.000016,-0.000005,-2.538441e-06,1.0,Binomial theorem,Addition,4,0.035118,0.035118,-54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,2048,51.0,442.0,234.0,15414.0,6.0,4.0,10.0,0.0,1.0,...,-0.000061,-0.001267,-9.993580e-05,1.0,Limit (mathematics),Mathematics,913,1.000000,1.000000,-49
914,2050,151.0,79.0,267.0,114.0,1.0,6.0,37.0,0.0,1.0,...,-0.000052,0.000011,1.894251e-06,0.0,Sine,Inverse trigonometric functions,914,0.000000,0.000000,1
915,2051,157.0,144.0,182.0,690.0,3.0,4.0,12.0,0.0,1.0,...,0.000016,-0.000017,-5.094307e-06,0.0,Quadratic equation,Rational number,915,0.022569,0.022569,-4
916,2053,82.0,198.0,207.0,660.0,3.0,2.0,14.0,1.0,0.0,...,-0.000054,-0.000047,-5.103651e-06,1.0,Summation,Arithmetic,916,0.047547,0.047547,-34


# Data Normalization

In [10]:
df = all_data[["relation", "tfidf_score", "rank", "f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
            "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
            "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
            "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37"]]

In [11]:
df.corr()

,relation,tfidf_score,rank,f0,f1,f2,f3,f4,f5,f6,...,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37
relation,1.000000,0.422548,-0.429074,-0.507082,0.632756,-0.300031,0.480312,-0.040273,0.080495,0.047215,...,0.280074,-0.181337,0.385592,-0.250013,-0.470336,0.580935,-0.007058,-0.451984,-0.501740,-0.533367
tfidf_score,0.422548,1.000000,-0.304845,-0.250265,0.510836,-0.158857,0.721204,0.017262,0.232769,0.009894,...,0.093243,-0.034417,0.160340,-0.079277,-0.240101,0.440513,-0.036652,-0.184324,-0.579303,-0.577687
rank,-0.429074,-0.304845,1.000000,0.328530,-0.338392,0.261637,-0.260353,-0.040442,0.013431,-0.013150,...,-0.125215,0.128138,-0.089580,0.099215,0.259491,-0.285568,-0.027814,0.240522,0.330653,0.352334
f0,-0.507082,-0.250265,0.328530,1.000000,-0.382509,0.676468,-0.240180,0.095387,-0.035923,0.257944,...,-0.041645,0.088662,-0.252775,0.115587,0.928859,-0.350094,0.299235,0.391122,0.585890,0.622036
f1,0.632756,0.510836,-0.338392,-0.382509,1.000000,-0.227941,0.696594,-0.066631,0.123986,0.237176,...,0.090792,-0.058945,0.121370,-0.257916,-0.356378,0.929387,0.248437,-0.406547,-0.589019,-0.627694
f2,-0.300031,-0.158857,0.261637,0.676468,-0.227941,1.000000,-0.150245,0.258493,-0.023567,-0.009630,...,-0.114425,-0.044844,-0.145258,-0.034110,0.522271,-0.211653,-0.048268,0.219335,0.757134,0.757451
f3,0.480312,0.721204,-0.260353,-0.240180,0.696594,-0.150245,1.000000,-0.049489,0.281859,-0.010191,...,-0.048614,-0.115277,-0.034111,-0.151737,-0.230355,0.572920,-0.044731,-0.246696,-0.757126,-0.756051
f4,-0.040273,0.017262,-0.040442,0.095387,-0.066631,0.258493,-0.049489,1.000000,-0.049874,-0.066085,...,0.053694,-0.063239,-0.019075,-0.034467,0.120122,-0.075579,0.008898,0.118274,0.206465,0.202002
f5,0.080495,0.232769,0.013431,-0.035923,0.123986,-0.023567,0.281859,-0.049874,1.000000,-0.041237,...,-0.055044,0.067800,-0.016307,-0.011269,-0.053665,0.163680,0.028162,-0.093185,-0.203019,-0.200688
f6,0.047215,0.009894,-0.013150,0.257944,0.237176,-0.009630,-0.010191,-0.066085,-0.041237,1.000000,...,0.267678,0.263849,0.125911,0.127103,0.250012,0.239174,0.664533,-0.007993,0.000465,0.000511


In [12]:
columns1 = ["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
            "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
            "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
            "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37"]

def normalize_array(x):
    x = np.array(x)
    min_x = np.min(x)
    max_x = np.max(x)
    x = (x - min_x)/(max_x - min_x)
    return x


def normalized_df(df):
    X = [normalize_array([df[[col]].values[i][0] for i in range(len(df[[col]]))]) for col in columns1]
    X = np.array(X)
    return X.T

X_sup = all_data[["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
            "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
            "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
            "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37"]]
X_sup = normalized_df(X_sup)

In [13]:
columns = ["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
            "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
            "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
            "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "rank", "wiki_tfidf_score"]

def normalize_array(x):
    x = np.array(x)
    min_x = np.min(x)
    max_x = np.max(x)
    x = (x - min_x)/(max_x - min_x)
    return x


def normalized_df(df):
    X = [normalize_array([df[[col]].values[i][0] for i in range(len(df[[col]]))]) for col in columns]
    X = np.array(X)
    return X.T

X = all_data[["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
            "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
            "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
            "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "rank", "wiki_tfidf_score"]]
X = normalized_df(X)

In [14]:
y = all_data[["relation"]].to_numpy().ravel()

# Data Modeling along with our features

In [15]:
def print_results(result):
    print("Precision: %.1f" % result[0])
    print("Recall: %.1f" % result[1])
    print("F1 Score: %.1f" % result[2])
    print("Area Under Curve: %.1f" % result[3])

def evaluation_results(y_test, y_predict):
    recall = recall_score(y_test, y_predict)*100
    precision = precision_score(y_test, y_predict)*100
    f1 = f1_score(y_test, y_predict)*100
    auc = roc_auc_score(y_test, y_predict)*100
    return [precision, recall, f1, auc]


def model_train(x_train, x_test, y_train, y_test, model):
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    return evaluation_results(y_test, y_predict)


def k_fold_training(X, Y, model):
    results = []
    kf = KFold(n_splits = 5)
    kf.get_n_splits(X)
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        result = model_train(x_train, x_test, y_train, y_test, model)
        results.append(result)
    results = np.array(results)
    final_result = np.mean(results, axis = 0)
    return final_result

## Supervised with Old Features

In [31]:
rand_params = {
    "n_estimators" : 200,
    "max_features" : 'auto',
}
model = RandomForestClassifier(**rand_params)
result = k_fold_training(X_sup, y, model)
print_results(result)

Precision: 90.5
Recall: 91.3
F1 Score: 90.9
Area Under Curve: 92.9


In [20]:
model = GaussianNB()
result = k_fold_training(X_sup, y, model)
print_results(result)

print("------------------")

model = LogisticRegression(solver='lbfgs')
result = k_fold_training(X_sup, y, model)
print_results(result)

print("------------------")


model = LinearSVC(random_state=0, tol=1e-5)
result = k_fold_training(X_sup, y, model)
print_results(result)

Precision: 81.1
Recall: 78.1
F1 Score: 79.2
Area Under Curve: 83.6
------------------
Precision: 86.2
Recall: 81.9
F1 Score: 83.9
Area Under Curve: 87.1
------------------
Precision: 88.6
Recall: 86.1
F1 Score: 87.2
Area Under Curve: 89.9


## Supervised with New Features

In [36]:
rand_params = {
    "n_estimators" : 200,
    "max_features" : 'auto',
}
model = RandomForestClassifier(**rand_params)
result = k_fold_training(X, y, model)
print_results(result)

Precision: 90.9
Recall: 90.3
F1 Score: 90.5
Area Under Curve: 92.5


In [33]:
model = GaussianNB()
result = k_fold_training(X, y, model)
print_results(result)

print("------------------")

model = LogisticRegression(solver='lbfgs')
result = k_fold_training(X, y, model)
print_results(result)

print("------------------")


model = LinearSVC(random_state=0, tol=1e-5)
result = k_fold_training(X, y, model)
print_results(result)

Precision: 81.1
Recall: 76.3
F1 Score: 78.3
Area Under Curve: 82.9
------------------
Precision: 85.9
Recall: 83.2
F1 Score: 84.4
Area Under Curve: 87.6
------------------
Precision: 89.0
Recall: 87.5
F1 Score: 88.2
Area Under Curve: 90.6


In [46]:
X

array([[0.57373272, 0.78571429, 0.05164472, ..., 0.49631068, 0.45045045,
        0.0052121 ],
       [0.11290323, 0.43778802, 0.00175177, ..., 0.46698469, 0.22522523,
        0.        ],
       [0.42626728, 0.34331797, 0.03730617, ..., 0.51191299, 0.48648649,
        0.1515644 ],
       ...,
       [0.34331797, 0.31336406, 0.01174333, ..., 0.47508084, 0.45045045,
        0.01830465],
       [0.17050691, 0.43778802, 0.01336534, ..., 0.47503513, 0.18018018,
        0.03856287],
       [0.42626728, 0.07373272, 0.03730617, ..., 0.51597097, 0.53153153,
        0.        ]])

In [49]:
result = k_fold_training(X, y)
print_results(result)

Precision: 90.4
Recall: 90.5
F1 Score: 90.3
Area Under Curve: 92.4


In [51]:
result = k_fold_training(X_sup, y)
print_results(result)

Precision: 89.8
Recall: 90.1
F1 Score: 89.9
Area Under Curve: 92.2


In [36]:
X.shape, X_sup.shape

((918, 40), (918, 38))

In [163]:
result = k_fold_training(X, y)
print_results(result)

Precision: 89.5
Recall: 88.3
F1 Score: 88.8
Area Under Curve: 91.4


In [156]:
result = k_fold_training(X, y)
print_results(result)

Precision: 89.0
Recall: 88.7
F1 Score: 88.8
Area Under Curve: 91.5


In [18]:
result = k_fold_training(X, y)
print_results(result)

Precision: 89.8
Recall: 88.2
F1 Score: 89.0
Area Under Curve: 91.5


In [16]:
result = k_fold_training(X, y)
print_results(result)

Precision: 94.4
Recall: 88.2
F1 Score: 91.2
Area Under Curve: 92.9
